
# Hierarchical Twitter Account Search with CrewAI

This notebook demonstrates a hierarchical agent setup using [CrewAI](https://github.com/crewAIInc/crewAI). The manager agent delegates search tasks to a specialist searcher agent. The searcher uses SerpAPI to find Twitter accounts related to a topic. The goal is to collect **1000 unique accounts**.

The hierarchical collaboration pattern follows the CrewAI documentation:

```python
from crewai import Agent, Crew, Task, Process
# Manager agent coordinates the team
manager = Agent(
    role="Project Manager",
    goal="Coordinate team efforts and ensure project success",
    backstory="Experienced project manager skilled at delegation and quality control",
    allow_delegation=True,
    verbose=True
)
# Specialist agents
researcher = Agent(
    role="Researcher",
    goal="Provide accurate research and analysis",
    backstory="Expert researcher with deep analytical skills",
    allow_delegation=False,
    verbose=True
)
# Hierarchical crew
crew = Crew(
    agents=[manager, researcher],
    tasks=[project_task],
    process=Process.hierarchical,
    manager_llm="gpt-4o",
    verbose=True
)
```



## Install dependencies


In [ ]:
!pip install crewai openai serpapi pandas -q


## Environment variables
Provide your API keys before running the agents.


In [ ]:
import os
SERPAPI_API_KEY = os.environ.get('SERPAPI_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
assert SERPAPI_API_KEY and OPENAI_API_KEY, 'Set SERPAPI_API_KEY and OPENAI_API_KEY environment variables.'


## SerpAPI search tool


In [ ]:
from typing import List, Dict
import serpapi
import pandas as pd

class TwitterSearchTool:
    def search(self, query: str, start: int = 0, num_results: int = 10) -> List[Dict]:
        params = {
            'q': f'site:twitter.com {query}',
            'api_key': SERPAPI_API_KEY,
            'start': start,
            'num': num_results
        }
        results = serpapi.search(params)
        accounts = []
        for item in results.get('organic_results', []):
            title = item.get('title')
            link = item.get('link')
            if link and 'twitter.com/' in link:
                accounts.append({'title': title, 'link': link})
        return accounts


## Helper function to gather accounts


In [ ]:
def gather_accounts(topic: str, batch_size: int = 10, target: int = 1000) -> pd.DataFrame:
    searcher = TwitterSearchTool()
    collected: List[Dict] = []
    start = 0
    while len(collected) < target:
        batch = searcher.search(topic, start=start, num_results=batch_size)
        if not batch:
            break
        for acc in batch:
            if acc not in collected:
                collected.append(acc)
        start += batch_size
        print(f'Collected {len(collected)} accounts')
    return pd.DataFrame(collected)


## Define CrewAI agents and tasks


In [ ]:
from crewai import Agent, Crew, Task, Process

manager = Agent(
    role='Search Manager',
    goal='Find 1000 Twitter accounts about a topic',
    backstory='Coordinates search strategy and delegates work',
    allow_delegation=True,
    verbose=True
)

searcher = Agent(
    role='Twitter Searcher',
    goal='Use SerpAPI to locate twitter accounts',
    backstory='Expert at advanced search queries',
    allow_delegation=False,
    verbose=True
)

search_task = Task(
    description='Gather 1000 unique Twitter accounts about the topic',
    expected_output='DataFrame with account titles and links',
    agent=manager
)

crew = Crew(
    agents=[manager, searcher],
    tasks=[search_task],
    process=Process.hierarchical,
    manager_llm='gpt-4o',
    verbose=True
)


## Execute search


In [ ]:
topic = 'AI news'
results_df = gather_accounts(topic)
print(results_df.head())